#🧩 1️⃣ Importação das bibliotecas


Importamos as bibliotecas necessárias para gerar e manipular os dados:


*   **pandas:** manipulação de tabelas de dados (DataFrames).
*   **random**: para gerar valores aleatórios, como produtos, cidades, clientes, quantidades e preços.
*   **datetime e timedelta:** para criar datas de venda realistas dentro de um período específico.
*   **numpy:** utilizado para cálculos vetorizados, como geração de preços ou quantidades com distribuição mais realista.

In [8]:
# Importando as bibliotecas:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta



# 🛍️ 2️⃣ Criação das listas de referência

Aqui definimos listas com possíveis valores para cada categoria:
- **Produtos** vendidos (ex: Camisa, Tênis, Jaqueta...)  
- **Cidades** onde as vendas ocorreram  
- **Vendedores** responsáveis pelas vendas  

Essas listas servirão de base para gerar os dados de forma aleatória.


In [10]:
#Criando colunas e registros
nomes_produtos = [
    "Camisa", "Calça", "Tênis", "Jaqueta",
    "Bolsa", "Relógio", "Boné", "Vestido"
]

cidades = [
    "São Paulo", "Rio de Janeiro", "Belo Horizonte",
    "Curitiba", "Porto Alegre", "Salvador", "Fortaleza"
]

vendedores = [
    "Marcos", "Ana", "João", "Carla", "Paulo",
    "Fernanda", "Lucas", "Patrícia"
]

# Dicionário de categorias (mapeia produto -> categoria)
categorias = {
    "Camisa": "Vestuário",
    "Calça": "Vestuário",
    "Vestido": "Vestuário",
    "Tênis": "Calçados",
    "Jaqueta": "Vestuário",
    "Bolsa": "Acessórios",
    "Relógio": "Acessórios",
    "Boné": "Acessórios"
}


# 📊 3️⃣ Geração dos dados de vendas


Neste bloco, criamos uma lista chamada dados que armazenará todas as linhas do nosso dataset de vendas:

- n_registros = 100: define que vamos gerar 100 vendas fictícias.

- Para cada venda:

       -id_venda: identificador único da venda (1 a 100).

       -produto: selecionado aleatoriamente da lista nomes_produtos.

       -categoria: obtida a partir do dicionário categorias; se o produto não estiver no dicionário, usamos "Outros".

       -cidade: cidade da venda, escolhida aleatoriamente da lista cidades.

       -vendedor: selecionado aleatoriamente da lista vendedores.

       -quantidade: quantidade vendida, gerada aleatoriamente entre 1 e 10.

       -preco_unitario: preço do produto, gerado aleatoriamente entre 50 e 500, arredondado para 2 casas decimais.

       -total_venda: calculado como quantidade * preco_unitario, arredondado para 2 casas decimais.

       -data_venda: data da venda, gerada aleatoriamente dentro do último ano a partir da data atual (datetime.now() - timedelta(days=random.randint(0, 365))).

Cada linha é adicionada à lista dados na forma de uma lista com todos esses campos, pronta para ser transformada em um DataFrame do pandas.


In [11]:
# Gerando 100 linhas de vendas fictícias (com ID, categoria e data)
dados = [] # Criando a lista
n_registros = 100 # Especificando o numero de registros

for i in range(n_registros):
    id_venda = i + 1
    produto = random.choice(nomes_produtos)
    categoria = categorias.get(produto, "Outros")
    cidade = random.choice(cidades)
    vendedor = random.choice(vendedores)
    quantidade = random.randint(1, 10)
    preco_unitario = round(random.uniform(50, 500), 2)
    total_venda = round(quantidade * preco_unitario, 2)
    # gerar datas ao longo do último ano
    data_venda = datetime.now() - timedelta(days=random.randint(0, 365))

    dados.append([
        id_venda,
        produto,
        categoria,
        cidade,
        vendedor,
        quantidade,
        preco_unitario,
        total_venda,
        data_venda.date()
    ])


#🧩 4️⃣ Inserção proposital de dados nulos, erros e duplicatas

Neste bloco, enriquecemos o dataset com inconsistências de forma proposital, para criar um cenário de dados “sujos” e permitir práticas de ETL mais realistas:

Primeiro, transformamos a lista dados em um DataFrame temporário (df_temp) para facilitar manipulações por coluna, mantendo depois o formato original como lista de listas.

- **1) Inserir valores nulos:**

Selecionamos 5 índices aleatórios e definimos o campo Vendedor como NaN.

Isso simula dados faltantes comuns em datasets reais.

- **2) Inserir erros de digitação:**

Selecionamos 3 índices aleatórios e modificamos a coluna Cidade, simulando erros como "São Paulo" → "Sao Pualo".

Isso cria dados inconsistentes, úteis para testar limpeza e padronização de strings.

- **3) Inserir preços inválidos:**

Selecionamos 2 registros e atribuímos valores de Preco_Unitario zero ou negativo.

Simula erros de entrada de dados que podem ocorrer em sistemas de vendas.

- **4) Inserir duplicatas propositalmente:**

Selecionamos 2 registros aleatórios e os duplicamos, simulando registros repetidos.

Permite testar a remoção de duplicatas durante o processo de ETL.



Por fim, atualizamos a variável dados convertendo o DataFrame de volta para lista de listas, mantendo o mesmo formato usado nas etapas seguintes do notebook.

In [12]:
# Transformar em DataFrame temporário para manipulação mais fácil
df_temp = pd.DataFrame(dados, columns=[
    "ID_Venda","Produto","Categoria","Cidade","Vendedor",
    "Quantidade","Preco_Unitario","Total_Venda","Data_Venda"
])

# 1) Inserir alguns valores nulos no Vendedor
null_indices = np.random.choice(df_temp.index, size=5, replace=False)
df_temp.loc[null_indices, "Vendedor"] = np.nan

# 2) Inserir alguns erros de digitação em 'Cidade' (simulando dados sujos)
erro_indices = np.random.choice(df_temp.index, size=3, replace=False)
df_temp.loc[erro_indices, "Cidade"] = df_temp.loc[erro_indices, "Cidade"].apply(
    lambda x: x.replace("São", "Sao").replace("Paulo", "Pualo") if isinstance(x, str) else x
)

# 3) Inserir um preço unitário zero ou negativo em 2 registros (erro de entrada)
preco_err_indices = np.random.choice(df_temp.index, size=2, replace=False)
df_temp.loc[preco_err_indices, "Preco_Unitario"] = [0, -99.9]

# 4) Opcional: duplicar propositalmente 2 registros para simular duplicatas
dup_indices = np.random.choice(df_temp.index, size=2, replace=False)
df_temp = pd.concat([df_temp, df_temp.loc[dup_indices]], ignore_index=True)

# Atualizar a variável 'dados' para continuar o fluxo (lista de listas)
dados = df_temp.values.tolist()



#🧩 5️⃣ Criação do DataFrame final e ajustes de tipos de dados


Neste bloco, organizamos o dataset final para análise e futuras etapas do ETL:


**Criar DataFrame final:**

- Transformamos a lista dados em um DataFrame df com colunas nomeadas corretamente.

- Isso garante consistência e clareza no dataset.

**Converter Data_Venda para datetime:**

- Garantimos que a coluna de datas esteja no formato datetime.

- Usamos errors="coerce" para transformar valores inválidos em NaT, evitando erros em análises temporais.

**Converter tipos numéricos explicitamente:**

- Quantidade: convertida para inteiro (Int64) com suporte a valores nulos.

- Preco_Unitario e Total_Venda: convertidos para numérico, permitindo cálculos futuros.

**Ordenar por data de venda:**

- Organiza o DataFrame cronologicamente, essencial para análises de séries temporais e relatórios de vendas.

- Resetamos o índice para manter a sequência correta após a ordenação.

In [13]:
# Criando o DataFrame final
df = pd.DataFrame(dados, columns=[
    "ID_Venda","Produto","Categoria","Cidade","Vendedor",
    "Quantidade","Preco_Unitario","Total_Venda","Data_Venda"
])

# Convertendo Data_Venda para datetime (se não estiver)
df["Data_Venda"] = pd.to_datetime(df["Data_Venda"], errors="coerce")

# Convertendo tipos numéricos
df["Quantidade"] = pd.to_numeric(df["Quantidade"], errors="coerce").astype("Int64")
df["Preco_Unitario"] = pd.to_numeric(df["Preco_Unitario"], errors="coerce")
df["Total_Venda"] = pd.to_numeric(df["Total_Venda"], errors="coerce")

# Ordenando por Data_Venda
df = df.sort_values(by="Data_Venda").reset_index(drop=True)


#🧩 6️⃣ Salvamento do CSV final e conferência do dataset

Neste bloco, finalizamos o processo de geração do dataset e salvamos o arquivo para uso futuro:

**Salvar o CSV:**

- to_csv("vendas.csv", index=False, encoding="utf-8-sig") salva o DataFrame em CSV, sem incluir o índice.

- A codificação utf-8-sig garante compatibilidade com Excel e sistemas que exigem BOM para UTF-8.

**Mensagem de confirmação:**

- Informamos que o dataset foi gerado com sucesso.

- Exibimos informações importantes para conferência rápida:

**Total de registros após inserção de nulos, erros e duplicatas.**

- Quantidade de registros com Vendedor nulo.

- Quantidade de registros com Preco_Unitario menor ou igual a zero.

**Visualização inicial:**

- display(df.head(10)) mostra as primeiras 10 linhas do DataFrame, permitindo verificar rapidamente a diversidade de dados e se os erros inseridos estão presentes.


In [14]:
# Salvando o CSV final
df.to_csv("vendas.csv", index=False, encoding="utf-8-sig")

# Mensagem final e visualização
print("✅ Dataset 'vendas.csv' gerado com sucesso!")
print(f"Total de registros (após injeções e duplicatas): {len(df)}")
print(f"Registros com vendedor nulo: {df['Vendedor'].isna().sum()}")
print(f"Registros com Preco_Unitario <= 0: {(df['Preco_Unitario'] <= 0).sum()}")
display(df.head(10))


✅ Dataset 'vendas.csv' gerado com sucesso!
Total de registros (após injeções e duplicatas): 102
Registros com vendedor nulo: 6
Registros com Preco_Unitario <= 0: 2


,ID_Venda,Produto,Categoria,Cidade,Vendedor,Quantidade,Preco_Unitario,Total_Venda,Data_Venda
0,32,Boné,Acessórios,Fortaleza,Patrícia,7,153.41,1073.87,2024-10-24
1,6,Boné,Acessórios,Salvador,Ana,9,493.45,4441.05,2024-10-30
2,47,Jaqueta,Vestuário,Belo Horizonte,João,3,445.31,1335.93,2024-11-02
3,19,Camisa,Vestuário,Fortaleza,Patrícia,7,50.78,355.46,2024-11-04
4,22,Camisa,Vestuário,Rio de Janeiro,Paulo,5,287.98,1439.90,2024-11-14
5,95,Jaqueta,Vestuário,Porto Alegre,Ana,1,176.35,176.35,2024-11-16
6,55,Tênis,Calçados,São Paulo,Lucas,9,307.40,2766.60,2024-11-17
7,90,Calça,Vestuário,Salvador,Fernanda,2,225.91,451.82,2024-11-19
8,5,Bolsa,Acessórios,Belo Horizonte,Lucas,8,50.95,407.60,2024-11-20
9,56,Relógio,Acessórios,Fortaleza,Fernanda,10,255.38,2553.80,2024-12-05


**### Próximos passos (para o notebook 02_pipeline_ETL.ipynb)**


- Ler `vendas.csv`.
- Tratar nulos em `Vendedor` (preencher com "Desconhecido" ou imputação).
- Corrigir erros de digitação em `Cidade` (string matching/manual rules).
- Tratar `Preco_Unitario <= 0` (corrigir ou remover).
- Remover duplicatas por `ID_Venda`.
- Ajustar tipos e criar coluna de dimensão (por exemplo, `Ano`, `Mes`).
- Salvar `vendas_tratadas.csv` e carregar em um banco SQLite (opcional).
